# PHM 데이터셋 고장 분류

CNN-LSTM 모델을 사용하여 장비 고장 유형을 분류하는 실습 자료입니다.

## 목표
- 시계열 센서 데이터 전처리 방법 학습
- 슬라이딩 윈도우 기법 이해 및 구현
- CNN-LSTM 하이브리드 모델 구축
- 고장 분류 모델 학습 및 평가

## 1. 라이브러리 Import

In [ ]:
# 기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# 딥러닝
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# 시드 고정 (재현성)
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)

## 2. 합성 데이터 생성

실제 PHM 데이터셋을 사용할 수 없는 경우를 대비해 합성 데이터를 생성합니다.

**Note**: 실제 프로젝트에서는 PHM Society Data Challenge에서 제공하는 실제 데이터를 사용하세요.

In [ ]:
def generate_synthetic_data(n_samples=10000, n_sensors=10, n_fault_types=4):
    """
    합성 PHM 센서 데이터 생성
    
    Parameters:
    - n_samples: 생성할 샘플 수
    - n_sensors: 센서 개수
    - n_fault_types: 고장 유형 수 (0: Normal, 1~3: Fault types)
    
    Returns:
    - df: 생성된 데이터프레임
    """
    
    # Hint: 각 고장 유형에 따라 다른 패턴을 가진 센서 데이터를 생성하세요
    # Normal 데이터는 낮은 분산, Fault 데이터는 높은 분산 또는 트렌드를 가지도록 설정
    
    data = []
    
    for i in range(n_samples):
        # TODO: 고장 유형을 무작위로 선택
        fault_type = None  # 0 ~ n_fault_types-1 중 하나를 선택
        
        # TODO: 고장 유형에 따라 다른 특성을 가진 센서 데이터 생성
        if fault_type == 0:  # Normal
            # 정상 상태: 평균 근처의 작은 변동
            sensor_data = None  # numpy를 사용하여 정규분포 데이터 생성
        else:  # Fault
            # 고장 상태: 더 큰 변동 또는 평균 shift
            sensor_data = None  # 고장 유형에 따라 다른 패턴 생성
        
        # 데이터 저장
        row = list(sensor_data) + [fault_type]
        data.append(row)
    
    # 데이터프레임 생성
    columns = [f'sensor_{i+1}' for i in range(n_sensors)] + ['fault_type']
    df = pd.DataFrame(data, columns=columns)
    
    return df

# 합성 데이터 생성 (실습용)
# df = generate_synthetic_data(n_samples=10000, n_sensors=10, n_fault_types=4)

# 실제 PHM 데이터를 사용하는 경우:
# df = pd.read_csv('../data/raw/phm_data.csv')
# PHM Society Data Challenge에서 다운로드한 데이터를 사용하세요

print("데이터 생성 함수 정의 완료")
# print(f"데이터 shape: {df.shape}")
# print(f"\n고장 유형 분포:\n{df['fault_type'].value_counts()}")

## 3. 데이터 탐색 (EDA)

**Hint**: 데이터의 분포, 고장 유형별 특성, 센서 간 상관관계 등을 시각화하세요.

In [ ]:
# TODO: 데이터프레임의 기본 정보 확인
# df.head(), df.info(), df.describe() 등을 활용

# TODO: 고장 유형별 분포 시각화
# plt.figure()를 사용하여 bar plot 생성

# TODO: 센서 데이터의 분포 확인
# 고장 유형별로 센서 값의 차이를 box plot 또는 histogram으로 시각화

pass

## 4. 데이터 전처리

### 4.1 특징(X)과 타겟(y) 분리

In [ ]:
# TODO: 센서 데이터(X)와 고장 유형(y)을 분리
# X = df.drop('fault_type', axis=1).values
# y = df['fault_type'].values

# print(f"X shape: {X.shape}")
# print(f"y shape: {y.shape}")

### 4.2 데이터 정규화

**Hint**: StandardScaler를 사용하여 각 센서 데이터를 정규화하세요. 이는 학습 속도와 성능을 향상시킵니다.

In [ ]:
# TODO: StandardScaler를 사용하여 데이터 정규화
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# print(f"정규화 후 평균: {X_scaled.mean():.4f}")
# print(f"정규화 후 표준편차: {X_scaled.std():.4f}")

### 4.3 슬라이딩 윈도우 생성

시계열 데이터를 고정 길이의 윈도우로 분할합니다.

**Hint**: 
- `window_size`: 한 샘플에 포함될 시간 스텝 수
- `stride`: 윈도우를 이동시키는 간격
- 각 윈도우의 마지막 시점의 레이블을 사용

In [ ]:
def create_sliding_windows(X, y, window_size=100, stride=50):
    """
    슬라이딩 윈도우를 사용하여 시계열 데이터를 변환
    
    Parameters:
    - X: 입력 데이터 (n_samples, n_features)
    - y: 레이블 (n_samples,)
    - window_size: 윈도우 크기
    - stride: 윈도우 이동 간격
    
    Returns:
    - X_windows: (n_windows, window_size, n_features)
    - y_windows: (n_windows,)
    """
    
    X_windows = []
    y_windows = []
    
    # TODO: 슬라이딩 윈도우 구현
    # for 루프를 사용하여 stride 간격으로 윈도우를 생성
    # 각 윈도우는 X[i:i+window_size]의 형태
    # 레이블은 윈도우의 마지막 시점 y[i+window_size-1] 사용
    
    for i in range(0, len(X) - window_size + 1, stride):
        # 윈도우 추출
        window = None  # TODO: X에서 윈도우 크기만큼 슬라이싱
        label = None   # TODO: 윈도우의 마지막 시점 레이블
        
        X_windows.append(window)
        y_windows.append(label)
    
    # numpy array로 변환
    X_windows = np.array(X_windows)
    y_windows = np.array(y_windows)
    
    return X_windows, y_windows

# 윈도우 생성
# window_size = 100
# stride = 50

# X_windows, y_windows = create_sliding_windows(X_scaled, y, window_size, stride)

# print(f"윈도우 데이터 shape: {X_windows.shape}")
# print(f"윈도우 레이블 shape: {y_windows.shape}")

### 4.4 One-hot Encoding

**Hint**: to_categorical을 사용하여 레이블을 one-hot 벡터로 변환하세요.

In [ ]:
# TODO: 레이블을 one-hot encoding으로 변환
# y_categorical = to_categorical(y_windows)

# print(f"One-hot 레이블 shape: {y_categorical.shape}")
# print(f"클래스 개수: {y_categorical.shape[1]}")

### 4.5 Train/Test Split

In [ ]:
# TODO: 학습/테스트 데이터 분리 (80:20)
# X_train, X_test, y_train, y_test = train_test_split(...)

# print(f"학습 데이터: {X_train.shape}, {y_train.shape}")
# print(f"테스트 데이터: {X_test.shape}, {y_test.shape}")

## 5. CNN-LSTM 모델 구축

### 모델 아키텍처
1. **Conv1D**: 센서 간 공간적 특징 추출
2. **LSTM**: 시간적 패턴 학습
3. **Dense**: 분류

**Hint**: 
- Input shape: (window_size, n_features)
- Conv1D 레이어 후 MaxPooling 적용
- LSTM 레이어는 return_sequences=False로 설정
- 최종 Dense 레이어는 softmax 활성화 함수 사용

In [ ]:
def build_cnn_lstm_model(input_shape, n_classes):
    """
    CNN-LSTM 모델 구축
    
    Parameters:
    - input_shape: (window_size, n_features)
    - n_classes: 분류할 클래스 개수
    
    Returns:
    - model: Keras 모델
    """
    
    model = models.Sequential()
    
    # TODO: CNN 레이어 추가
    # Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape)
    # MaxPooling1D(pool_size=2)
    # Conv1D(filters=128, kernel_size=3, activation='relu')
    # MaxPooling1D(pool_size=2)
    
    # TODO: LSTM 레이어 추가
    # LSTM(units=100, return_sequences=False)
    # Dropout(0.3) - 과적합 방지
    
    # TODO: Dense 레이어 추가
    # Dense(units=50, activation='relu')
    # Dropout(0.3)
    # Dense(units=n_classes, activation='softmax')  # 최종 출력
    
    return model

# 모델 생성
# input_shape = (window_size, X_scaled.shape[1])
# n_classes = y_categorical.shape[1]

# model = build_cnn_lstm_model(input_shape, n_classes)

# TODO: 모델 컴파일
# optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']

# model.summary()

## 6. 모델 학습

**Hint**: 
- EarlyStopping: val_loss가 개선되지 않으면 조기 종료
- ModelCheckpoint: 최고 성능 모델 저장
- ReduceLROnPlateau: 학습률 동적 조정

In [ ]:
# TODO: 콜백 설정
# early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# model_checkpoint = ModelCheckpoint('../models/best_model.h5', save_best_only=True)
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# callbacks = [early_stopping, model_checkpoint, reduce_lr]

# TODO: 모델 학습
# history = model.fit(
#     X_train, y_train,
#     validation_split=0.2,
#     epochs=50,
#     batch_size=64,
#     callbacks=callbacks,
#     verbose=1
# )

## 7. 학습 결과 시각화

In [ ]:
# TODO: 학습 곡선 시각화 (Loss와 Accuracy)
# plt.figure(figsize=(14, 5))

# Subplot 1: Loss
# plt.subplot(1, 2, 1)
# plt.plot(history.history['loss'], label='Train Loss')
# plt.plot(history.history['val_loss'], label='Val Loss')
# plt.legend(), plt.title('Loss'), plt.xlabel('Epoch'), plt.ylabel('Loss')

# Subplot 2: Accuracy
# plt.subplot(1, 2, 2)
# plt.plot(history.history['accuracy'], label='Train Accuracy')
# plt.plot(history.history['val_accuracy'], label='Val Accuracy')
# plt.legend(), plt.title('Accuracy'), plt.xlabel('Epoch'), plt.ylabel('Accuracy')

# plt.show()

## 8. 모델 평가

**Hint**: 테스트 데이터로 모델 성능을 평가하고, Confusion Matrix와 Classification Report를 출력하세요.

In [ ]:
# TODO: 테스트 데이터로 예측
# y_pred_prob = model.predict(X_test)
# y_pred = np.argmax(y_pred_prob, axis=1)
# y_true = np.argmax(y_test, axis=1)

# TODO: 정확도 계산
# accuracy = accuracy_score(y_true, y_pred)
# print(f"테스트 정확도: {accuracy:.4f}")

# TODO: Classification Report
# print("\nClassification Report:")
# print(classification_report(y_true, y_pred, target_names=[f'Class {i}' for i in range(n_classes)]))

### 8.1 Confusion Matrix 시각화

In [ ]:
# TODO: Confusion Matrix 계산 및 시각화
# cm = confusion_matrix(y_true, y_pred)

# plt.figure(figsize=(8, 6))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
# plt.title('Confusion Matrix')
# plt.ylabel('True Label')
# plt.xlabel('Predicted Label')
# plt.show()

## 9. 결과 분석 및 개선 방향

**질문**:
1. 모델의 성능이 만족스러운가요?
2. 어떤 클래스가 가장 잘 분류되었나요?
3. 어떤 클래스가 혼동되기 쉬운가요?
4. 성능을 개선하기 위해 시도할 수 있는 방법은?
   - 데이터 증강
   - 하이퍼파라미터 튜닝 (window_size, stride, LSTM units, CNN filters)
   - Attention mechanism 추가
   - 클래스 불균형 처리 (class_weight, SMOTE)

**실습 과제**:
- 윈도우 크기를 변경해보고 성능 변화를 관찰하세요
- LSTM units 수를 조정해보세요
- 추가 Conv1D 레이어를 추가하거나 제거해보세요

In [ ]:
# 여기에 추가 실험 코드를 작성하세요


## 10. 모델 저장 및 로드

In [ ]:
# TODO: 최종 모델 저장
# model.save('../models/final_cnn_lstm_model.h5')
# print("모델 저장 완료")

# TODO: 모델 로드 (필요시)
# loaded_model = keras.models.load_model('../models/final_cnn_lstm_model.h5')
# print("모델 로드 완료")

## 완료!

수고하셨습니다. 이 노트북을 통해 CNN-LSTM 모델을 사용한 고장 분류의 전체 파이프라인을 학습하셨습니다.

추가 학습을 위해 solution 노트북을 참고하세요.